
# POC 激活验证

### 1. 同构激活图

已完成

### 2. 设计传播函数模拟神经网络传参
- 图片分类网络结果  
    - [x] 读取数据
    - [x] 调用分类网络获得分类结果
    - [x] 获取网络中间结果  
- 设计传播函数  
    - [ ] conv 层
    - [ ] 全连接层
    - [ ] 输出层获得分类结果

In [2]:
import dgl
import torch
from utils_gnn import cnn2graph_activation
# from model_lib import mnist_cnn_model as father_model
from utils_basic import load_spec_model
from utils_gnn import padding
x = '/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models5/shadow_jumbo_9.model'
# load model 
# Model = load_spec_model(father_model, '5')
from model_lib.mnist_cnn_model import Model6 as Model
model = Model(gpu=True)
params = torch.load(x)
model.load_state_dict(params)

# load model detail 
model_detail = {}
model_detail_path = "./intermediate_data/model_detail.json"
import json
with open(model_detail_path, 'r') as f:
    model_detail = json.load(f)
print(model_detail)
g = cnn2graph_activation(model, model_detail['mnist']['5'])
dgl.save_graphs('./intermediate_data/grapj_test.bin', g)

/home/dorian/.conda/envs/mntd/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'mnist': {'0': [{'name': 'conv1', 'num': 16, 'maxpool': {'kernel_size': 2, 'stride': 2, 'padding': 0, 'dilation': 1, 'ceil_mode': False}}, {'name': 'conv2', 'num': 32, 'maxpool': {'kernel_size': 2, 'stride': 2, 'padding': 0, 'dilation': 1, 'ceil_mode': False}}, {'name': 'fc', 'num': 1, 'maxpool': None}, {'name': 'output', 'num': 1, 'maxpool': None}], '1': [{'name': 'conv1', 'num': 16, 'maxpool': {'kernel_size': 2, 'stride': 1, 'padding': 0, 'dilation': 1, 'ceil_mode': False}}, {'name': 'conv2', 'num': 32, 'maxpool': {'kernel_size': 2, 'stride': 1, 'padding': 0, 'dilation': 1, 'ceil_mode': False}}, {'name': 'conv3', 'num': 32, 'maxpool': {'kernel_size': 2, 'stride': 1, 'padding': 0, 'dilation': 1, 'ceil_mode': False}}, {'name': 'fc', 'num': 1, 'maxpool': None}, {'name': 'output', 'num': 1, 'maxpool': None}], '2': [{'name': 'conv1', 'num': 32, 'maxpool': None}, {'name': 'conv2', 'num': 32, 'maxpool': None}, {'name': 'conv3', 'num': 64, 'maxpool': None}, {'name': 'fc', 'num': 1, 'maxpool

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
# from utils_gnn import SGNACT
GPU = True
if GPU:
        torch.cuda.manual_seed_all(0)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
transform = transforms.Compose([
            transforms.ToTensor(),
        ])
BATCH_SIZE = 1
# MNIST image dataset 
trainset = torchvision.datasets.MNIST(root='./raw_data/', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)

from model_lib.hetero_struct_backdoor_dataset import HeteroStrucBackdoorDataset
from dgl.dataloading import GraphDataLoader
# Shadow model dataset
dataset = HeteroStrucBackdoorDataset(nums=10,
    raw_dir='/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models', activation=True)
model_dataloader = GraphDataLoader(dataset, batch_size=1, pin_memory=torch.cuda.is_available())



In [4]:
import dgl
from dgl.data import DGLDataset
import os
from tqdm import tqdm
from dgl import save_graphs, load_graphs
import torch
import json
from torch import nn
import numpy as np
# from model_lib.mnist_cnn_model import Model
from random import randint
from utils_basic import load_spec_model
from sklearn.metrics import roc_auc_score
from torchinfo import summary
import torch.nn.functional as F
from dgl.nn.pytorch.conv import GINConv
from dgl.nn.pytorch.glob import SumPooling, AvgPooling, MaxPooling, SortPooling
from utils_gnn import MLP
from utils_gnn import unpadding, padding


def conv(data, weight, bias, kernel_size, stride, padding):
    row, col = weight.size()
    # get actual conv kernel weight and bias
    w = unpadding(weight, kernel_size, kernel_size)
    w = w.unsqueeze(0).unsqueeze(0)
    b = unpadding(bias, 1, 1)[0]
    # get conv operator 
    operator = torch.nn.Conv2d(1, 1, kernel_size=kernel_size, stride=stride,
                    padding=padding)
    # set conv operator weight and bias
    operator.weight.data = w
    operator.bias.data = b
    # conduct conv operation
    x = operator(data)
    return padding(x[0][0], 512, 512)

def maxpool(kernel_size, stride, padding):
    pass

def message_func(edges):
    print("this is message function.")
    print("message fucntion ends")
    return {'pre': edges.src['x']}

def reduce_func(nodes):
    print("this is reduce function")
    input_mask = nodes.data['tag'] == 0
    conv_mask = nodes.data['tag'] == 1
    concat_mask = nodes.data['tag'] == 2
    fc_mask = nodes.data['tag'] == 3

    # input_feat = nodes.data['x'][input_mask]
    # print(input_feat)
    print("reduce function ends")
    return {'h': nodes.data['x']}


def cnn_cal(graph):
    graph.update_all(message_func=message_func, reduce_func=reduce_func)
    ft = graph.ndata['ft'][0]
    return ft

model.eval()
for batch, (batched_graph, labels) in enumerate(model_dataloader):
    print(batched_graph)
    for i, (x_in, y_in) in enumerate(dataloader):
        # print(x_in.size())
        # get orginal cnn calculation resutls
        pred, params = model(x_in)
        print(torch.argmax(pred))
        # get calculation results on graph node 
        print(batched_graph.ndata.keys())
        res = cnn_cal(batched_graph)
        # compare
        break
    break

{'kernel_size': 2, 'stride': 2, 'padding': 0, 'dilation': 1, 'ceil_mode': False}
None
None
{'kernel_size': 2, 'stride': 2, 'padding': 0, 'dilation': 1, 'ceil_mode': False}
None
None
Graph(num_nodes=162, num_edges=5153,
      ndata_schemes={'x': Scheme(shape=(512, 512), dtype=torch.float32), 'tag': Scheme(shape=(), dtype=torch.int64), 'layer': Scheme(shape=(), dtype=torch.int64), 'node_size': Scheme(shape=(2,), dtype=torch.int64), 'params': Scheme(shape=(3, 2), dtype=torch.int64), 'bias': Scheme(shape=(1, 512), dtype=torch.float32), 'bias_size': Scheme(shape=(2,), dtype=torch.int64), 'pooling': Scheme(shape=(5,), dtype=torch.int64)}
      edata_schemes={})
tensor(3, device='cuda:0')
dict_keys(['x', 'tag', 'layer', 'node_size', 'params', 'bias', 'bias_size', 'pooling'])
this is message function.
message fucntion ends


RuntimeError: CUDA out of memory. Tried to allocate 5.03 GiB (GPU 0; 3.82 GiB total capacity; 256.19 MiB already allocated; 2.08 GiB free; 258.00 MiB reserved in total by PyTorch)

In [54]:
mask = g.ndata["tag"] == 0
g.ndata['test'] = torch.tensor([1. if i else 0 for i in mask]).to("cuda")

In [55]:
print(g.ndata["test"])

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.], device='cuda:0')


In [56]:
def message_func(edges):
    print("message function called.")
    return {"m": edges.src['test']}
from dgl import function as fn
reduce_func = fn.max("m", "h")
# def reduce_func(nodes):
#     print("reduce function called.")
#     return {"h": torch.max(nodes.mailbox["m"], dim=1)}

In [ ]:
while g.ndata['ft'][-1]:
    pass

In [58]:
g.update_all(message_func, reduce_func)
print(g.ndata["h"])

message function called.
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.], device='cuda:0')


In [70]:
g.ndata["test"] = g.ndata.pop("h")
g.update_all(message_func, reduce_func)
print(g.ndata['h'])

message function called.
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')
